# IIC2115 - Programación Como Herramienta para la Ingeniería
## Taller 3a
### Ayudante: Matías Gaete Silva - mzgaete@uc.cl
Importamos librerías necesarias:

In [19]:
import json
import sqlite3

#### Misión 1
Leemos la base de datos y visualizamos su estructura:

In [20]:
with open('movies.json', encoding='utf8') as movies_file:
    movies = json.load(movies_file)
movies

[{'title': 'After Dark in Central Park',
  'year': 1900,
  'cast': [],
  'genres': []},
 {'title': "Boarding School Girls' Pajama Parade",
  'year': 1900,
  'cast': [],
  'genres': []},
 {'title': "Buffalo Bill's Wild West Parad",
  'year': 1900,
  'cast': [],
  'genres': []},
 {'title': 'Caught', 'year': 1900, 'cast': [], 'genres': []},
 {'title': 'Clowns Spinning Hats', 'year': 1900, 'cast': [], 'genres': []},
 {'title': 'Capture of Boer Battery by British',
  'year': 1900,
  'cast': [],
  'genres': ['Short', 'Documentary']},
 {'title': 'The Enchanted Drawing', 'year': 1900, 'cast': [], 'genres': []},
 {'title': 'Feeding Sea Lions',
  'year': 1900,
  'cast': ['Paul Boyton'],
  'genres': []},
 {'title': 'How to Make a Fat Wife Out of Two Lean Ones',
  'year': 1900,
  'cast': [],
  'genres': ['Comedy']},
 {'title': 'New Life Rescue', 'year': 1900, 'cast': [], 'genres': []},
 {'title': 'New Morning Bath', 'year': 1900, 'cast': [], 'genres': []},
 {'title': 'Searching Ruins on Broadway, 

Nos conectamos a la base de datos ```movies.db``` (si no existe, entonces la crea). Creamos las tablas:
* Movies: tendrá un id, el título y año de la película.
* Actors: tendrá un id y nombre del actor/actriz.
* Genres: tendrá un id y nombre del género.
* ActorMovies: relaciona los actores con las películas.
* GenresMovies: relaciona los géneros con las películas.

Guardamos los cambios y cerramos la conexión.

In [21]:
connection = sqlite3.connect('movies.db')
cursor = connection.cursor()

cursor.execute("CREATE TABLE Movies(mid INTEGER PRIMARY KEY, title TEXT, year INTEGER)")
cursor.execute("CREATE TABLE Actors(aid INTEGER PRIMARY KEY, name TEXT)")
cursor.execute("CREATE TABLE Genres(gid INTEGER PRIMARY KEY, genre TEXT)")
cursor.execute("CREATE TABLE ActorsMovies(actor_id INTEGER, movie_id INTEGER, FOREIGN KEY (actor_id) REFERENCES Actors, FOREIGN KEY (movie_id) REFERENCES Movies)")
cursor.execute("CREATE TABLE GenresMovies(genre_id INTEGER, movie_id INTEGER, FOREIGN KEY (genre_id) REFERENCES Genres, FOREIGN KEY (movie_id) REFERENCES Movies)")

connection.commit()
connection.close()

#### Misión 2

Nos conectamos a la base creada. Cargamos las tablas creadas con los datos del archivo ``movies.json``. Guardamos los cambios y cerramos la conexión.

In [22]:
connection = sqlite3.connect('movies.db')
cursor = connection.cursor()

mid = 1
aid = 1
gid = 1
genres = {}
actors = {}

for movie in movies:

    title = movie["title"]
    year = movie["year"]

    cursor.execute("INSERT INTO Movies VALUES (?,?,?)", (mid, title, year))

    for genre in movie["genres"]:
        if genre not in genres:
            genres[genre] = gid
            cursor.execute("INSERT INTO Genres VALUES (?,?)", (gid, genre))
            gid += 1
        cursor.execute("INSERT INTO GenresMovies VALUES (?,?)", (genres[genre], mid))

    for actor in movie["cast"]:
        if actor not in actors:
            actors[actor] = aid
            cursor.execute("INSERT INTO Actors VALUES (?,?)", (aid, actor))
            aid += 1
        cursor.execute("INSERT INTO ActorsMovies VALUES (?,?)", (actors[actor], mid))

    mid += 1

connection.commit()
connection.close()

**Consideración**: Cuando asignamos un INTEGER PRIMARY KEY a una columna de una tabla, si al insertar datos a la tabla no se indica un valor para esta columna, entonces sqlite automáticamente generará un id por defecto para esa columna. Se ejecutaría de esta manera:

In [23]:
connection = sqlite3.connect('movies.db')
cursor = connection.cursor()

mid = 1
aid = 1
gid = 1
genres = {}
actors = {}

for movie in movies:

    title = movie["title"]
    year = movie["year"]

    cursor.execute("INSERT INTO Movies(title,year) VALUES (?,?)", (title,year))

    for genre in movie["genres"]:
        if genre not in genres:
            genres[genre] = gid
            cursor.execute("INSERT INTO Genres(genre) VALUES (?)", (genre,))
            gid += 1
        cursor.execute("INSERT INTO GenresMovies VALUES (?,?)", (genres[genre],mid))

    for actor in movie["cast"]:
        if actor not in actors:
            actors[actor] = aid
            cursor.execute("INSERT INTO Actors(name) VALUES (?)", (actor,))
            aid += 1
        cursor.execute("INSERT INTO ActorsMovies VALUES (?,?)", (actors[actor],mid))

    mid += 1

connection.commit()
connection.close()